## Name and ID

Shaohong Luo 1408604

## HW05 Code


You will complete the following notebook, as described in the PDF for Homework 05 (included in the download with the starter code).  You will submit:
1. This notebook file, along with your COLLABORATORS.txt file and the two tree images (PDFs generated using `graphviz` within the code), to the Gradescope link for code.
2. A PDF of this notebook and all of its output, once it is completed, to the Gradescope link for the PDF.


Please report any questions to the [class Piazza page](https://piazza.com/tufts/spring2021/comp135).

### Import required libraries.

In [88]:
import numpy as np
import pandas as pd
import math
import sklearn.tree
import graphviz

## Decision Trees

You should start by computing the two heuristic values for the toy data described in the assignment handout. You should then load the two versions of the abalone data, compute the two heuristic values on features (for the simplified data), and then build decision trees for each set of data.

### 1 Compute both heuristics for toy data.

#### (a) Compute the counting-based heuristic, and order the features by it.

In [2]:
# TODO
feature_1 = 3/4
feature_2 = 3/4
print('feature_1:',feature_1)
print('feature_2:',feature_2)

feature_1: 0.75
feature_2: 0.75


#### (b) Compute the information-theoretic heuristic, and order the features by it.

In [3]:
# TODO
H_example = 1.0
#tree 1
h_1_1 = 0
h_1_2 = -(1/3*math.log(1/3,2)+2/3*math.log(2/3,2))
gain_1 = 1-(1/4*h_1_1 + 3/4*h_1_2)
print('feature_1:',gain_1)

feature_1: 0.31127812445913283


In [4]:
h_2_1 = -(3/4*math.log(3/4,2)+1/4*math.log(1/4,2))
h_2_2 = -(1/4*math.log(1/4,2)+3/4*math.log(3/4,2))
gain_2 = 1-(1/2*h_2_1 + 1/2*h_2_2)
print('feature_2:',gain_2)

feature_2: 0.18872187554086717


#### (c) Discussion of results.

The counting method cannot disguish the best feature between 1 and 2, but information-theoretic heuristic can. It means that the information-theoretic heuristic is more robost( it performs much better to avoid producing the same result.And information-theoretic heuristic is the better way to construct decision-tree.

### 2 Compute both heuristics for simplified abalone data.

In [25]:
data = pd.read_csv('small_binary_x_train.csv')
y = pd.read_csv('3class_y_train.csv')
data = pd.concat([data,y],axis=1)
data.head()

,is_male,length_mm,diam_mm,height_mm,rings
0,0,0,0,0,0
1,0,1,1,0,1
2,0,1,1,1,1
3,0,0,0,0,0
4,0,1,1,1,1


#### (a) Compute the counting-based heuristic, and order the features by it.

In [114]:
# TODO
features = ['is_male','length_mm','diam_mm','height_mm']
s_total = data.loc[:,'rings'].count()
for i in features:
    tree_true = data[data.loc[:,i]==1].loc[:,'rings'].count()
    tree_false = data[data.loc[:,i]==0].loc[:,'rings'].count()
    if tree_true>tree_false:
        print(i,":",tree_true/s_total)
    else:
        print(i,":",tree_false/s_total)



is_male : 0.6394836272040302
length_mm : 0.5585642317380353
diam_mm : 0.551007556675063
height_mm : 0.5431360201511335


#### (b) Compute the information-theoretic heuristic, and order the features by it.

In [79]:
# TODO
features = ['is_male','length_mm','diam_mm','height_mm']
answer=[]
s = data.loc[:,'rings'].value_counts()
s_total = data.loc[:,'rings'].count()
p_sm = s[0]/s_total
p_md = s[1]/s_total
p_lg = s[2]/s_total
h_abalone = -(p_sm*math.log(p_sm,2)+p_md*math.log(p_md,2)+p_lg*math.log(p_lg,2))
for i in features:
    tree_true = data[data.loc[:,i]==1].loc[:,'rings'].value_counts()
    tree_true_num = data[data.loc[:,i]==1].loc[:,'rings'].count()
    tree_true_small = tree_true[1]/tree_true_num
    tree_true_medium = tree_true[0]/tree_true_num
    tree_true_large = tree_true[2]/tree_true_num
    h_true = -(tree_true_small*math.log(tree_true_small,2)+tree_true_medium*math.log(tree_true_medium,2)+tree_true_large*math.log(tree_true_large,2))

    tree_false = data[data.loc[:,i]==0].loc[:,'rings'].value_counts()
    tree_false_num = data[data.loc[:,i]==0].loc[:,'rings'].count()
    tree_false_small = tree_false[0]/tree_false_num
    tree_false_medium = tree_false[1]/tree_false_num
    tree_false_large = tree_false[2]/tree_false_num
    h_false = -(tree_false_small*math.log(tree_false_small,2)+tree_false_medium*math.log(tree_false_medium,2)+tree_false_large*math.log(tree_false_large,2))

    tree_information_gain = h_abalone - ((tree_true_num/s_total) * h_true + (tree_false_num/s_total) * h_false)
    answer.append(tree_information_gain)
    
print('is_male:',answer[0])
print('length_mm:',answer[1])
print('diam_mm:',answer[2])
print('height_mm:',answer[3])

is_male: 0.024516482271752293
length_mm: 0.13543816377043683
diam_mm: 0.1500706886802703
height_mm: 0.17302867291002477


### 3 Generate decision trees for full- and restricted-feature data

#### (a) Print accuracy values and generate tree images.

In [91]:
x_train = pd.read_csv('x_train.csv')
y_train = pd.read_csv('y_train.csv')
x_test = pd.read_csv('x_test.csv')
y_test = pd.read_csv('y_test.csv')

x_simp_train = pd.read_csv('small_binary_x_train.csv')
y_simp_train = pd.read_csv('3class_y_train.csv')
x_simp_test = pd.read_csv('small_binary_x_test.csv')
y_simp_test = pd.read_csv('3class_y_test.csv')


In [92]:
# TODO
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(criterion='entropy')
decision_tree.fit(x_train,y_train)
decision_tree.score(x_test,y_test)

0.194

In [94]:
decision_tree2 = DecisionTreeClassifier(criterion='entropy')
decision_tree2.fit(x_simp_train,y_simp_train)
decision_tree2.score(x_simp_test,y_simp_test)

0.722

In [89]:
dot_data = sklearn.tree.export_graphviz(decision_tree, out_file=None)
graph = graphviz.Source(dot_data)
graph.render("abalone")

'abalone.pdf'

In [95]:
dot_data = sklearn.tree.export_graphviz(decision_tree2, out_file=None)
graph = graphviz.Source(dot_data)
graph.render("abalone-simplified")

'abalone-simplified.pdf'

#### (b) Discuss the results seen for the two trees
The tree constructed by the original data is quite complex, while the other constructed by the simplified data is more straigtforward. The reason of difference is that the simplified vision has much less values, it only takes '0','1' as the criteria, while the orginal data tree has much more data, which causes the complexity.  
Error: Underfitting. 
